# Transfer Learning with Keras


In this notebook, we will present an introduction to Transfer Learning which are a few simple yet effective methods that you can use to build a powerful image classifier, using only very few training examples --- just a few hundred or thousand pictures from each class you want to be able to recognize --- by utilizing pretrained models. 

We will go over the following options:

* training a small network from scratch (as a baseline)
* using the bottleneck features of a pre-trained network
* fine-tuning the top layers of a pre-trained network

By the end you will know how to use the following Keras APIs:

* `ImageDataGenerator` for real-time data augmentation
*  `fit_generator` for training a Keras model using Python data generators
* layer freezing and model fine-tuning
* ...and more!

## Cats-Dogs Dataset

We will start from a setup of only 2000 training examples (1000 per class either a dog or a cat). The data is stored in the `data/cats-dogs` folder which has a `train` data directory and a `validation` data directory containing one subdirectory per image class, filled with `.jpg` images:

```
cats-dogs/
    train/
        dogs/
            dog.001.jpg
            dog.002.jpg
            ...
        cats/
            cat.001.jpg
            cat.002.jpg
            ...
    validation/
        dogs/
            dog.003.jpg
            dog.004.jpg
            ...
        cats/
            cat.003.jpg
            cat.004.jpg
            ...
```

The `validataion` set consists of 400 additional samples from each class as validation data, to evaluate our models.

Note that `2000` samples are *very* few examples to learn from, for a classification problem that is far from simple. So this is a challenging machine learning problem, but it is also a realistic one: in a lot of real-world use cases, even small-scale data collection can be extremely expensive or sometimes near-impossible (e.g. in medical imaging). Being able to make the most out of very little data is a key skill of a competent data scientist.

In [ ]:
import os, shutil

In [ ]:
base_dir = os.path.join('..','..','data','cats-dogs')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [ ]:
print('total training cat images:', len(os.listdir(train_cats_dir)))

In [ ]:
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))

This is what the data looks like:

![](../../images/dogs-cats.jpg)


## Transfer Learning

You might have heard that  "deep learning is only relevant when you have a huge amount of data". While not entirely incorrect, this is somewhat misleading. Certainly, deep learning requires the ability to learn features automatically from the data, which is generally only possible when lots of training data is available --- especially for problems where the input samples are very high-dimensional, like images. However, convolutional neural networks --- a pillar algorithm of deep learning--- are by design one of the best models available for most "perceptual" problems (such as image classification), even with very little data to learn from. Training a convnet from scratch on a small image dataset will still yield reasonable results, without the need for any custom feature engineering. Convnets are just plain good. They are the right tool for the job.

But what's more, deep learning models are by nature highly repurposable: you can take, say, an image classification or speech-to-text model trained on a large-scale dataset then reuse it on a significantly different problem with only minor changes, as we will see in this tutorial. Specifically in the case of computer vision, many pre-trained models (usually trained on the ImageNet dataset) are now publicly available for download and can be used to bootstrap powerful vision models out of very little data.

## Data pre-processing and data augmentation

In order to make the most of our few training examples, we will "augment" them via a number of random transformations, so that our model would never see the exact same picture twice. This helps prevent overfitting and helps the model generalize better.

In Keras this can be done via the `keras.preprocessing.image.ImageDataGenerator` class. This class allows you to:

* configure random transformations and normalization operations to be done on your image data during training
* instantiate generators of augmented image batches (and their labels) via `.flow(data, labels)` or `.flow_from_directory(directory)`. 

These generators can then be used with the Keras model methods that accept data generators as inputs, `fit_generator`, `evaluate_generator` and `predict_generator`.

Let's look at an example right away:

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

These are just a few of the options available (for more, see [the documentation](http://keras.io/preprocessing/image/)). Let's quickly go over what we just wrote:

* `rotation_range` is a value in degrees (0-180), a range within which to randomly rotate pictures
* `width_shift` and `height_shift` are ranges (as a fraction of total width or height) within which to randomly translate pictures vertically or horizontally
* `rescale` is a value by which we will multiply the data before any other processing. Our original images consist in RGB coefficients in the 0-255, but such values would be too high for our models to process (given a typical learning rate), so we target values between 0 and 1 instead by scaling with a 1/255. factor.
* `shear_range` is for randomly applying [shearing transformations](https://en.wikipedia.org/wiki/Shear_mapping)
* `zoom_range` is for randomly zooming inside pictures
* `horizontal_flip` is for randomly flipping half of the images horizontally ---relevant when there are no assumptions of horizontal assymetry (e.g. real-world pictures).
* `fill_mode` is the strategy used for filling in newly created pixels, which can appear after a rotation or a width/height shift.

Now let's start generating some pictures using this tool and save them to a temporary directory, so we can get a feel for what our augmentation strategy is doing ---we disable rescaling in this case to keep the images displayable:

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [ ]:
from keras.preprocessing.image import img_to_array, load_img

img = load_img(os.path.join(train_cats_dir, 'cat.10008.jpg'))  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (374, 500, 3)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 374, 500, 3)

The `.flow()` command below generates batches of randomly transformed images and it will loop indefinitely, so we need to `break` the loop at some point:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

from keras.preprocessing import image

i = 0
for batch in datagen.flow(x, batch_size=1):
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i % 4 == 0:
        break  # otherwise the generator would loop indefinitely
plt.show()

## Baseline: A small ConvNet

The right tool for an image classification job is a convnet, so let's try to train one on our data, as an initial baseline. Since we only have few examples, our number one concern should be *overfitting*. Overfitting happens when a model exposed to too few examples learns patterns that do not generalize to new data, i.e. when the model starts using irrelevant features for making predictions. For instance, if you, as a human, only see three images of people who are lumberjacks, and three, images of people who are sailors, and among them only one lumberjack wears a cap, you might start thinking that wearing a cap is a sign of being a lumberjack as opposed to a sailor. You would then make a pretty lousy lumberjack/sailor classifier.

Data augmentation is one way to fight overfitting, but it isn't enough since our augmented samples are still highly correlated. Your main focus for fighting overfitting should be the entropic capacity of your model ---how much information your model is allowed to store. A model that can store a lot of information has the potential to be more accurate by leveraging more features, but it is also more at risk to start storing irrelevant features. Meanwhile, a model that can only store a few features will have to focus on the most significant features found in the data, and these are more likely to be truly relevant and to generalize better.

There are different ways to modulate entropic capacity. The main one is the choice of the number of parameters in your model, i.e. the number of layers and the size of each layer. Another way is the use of weight regularization, such as L1 or L2 regularization, which consists in forcing model weights to taker smaller values.

In our case we will use a very small convnet with few layers and few filters per layer, alongside data augmentation and dropout. Dropout also helps reduce overfitting, by preventing a layer from seeing the exact same pattern twice, thus acting in a way analoguous to data augmentation (you could say that both dropout and data augmentation tend to disrupt random correlations occuring in your data).

The code snippet below is our first model, a simple stack of 3 convolution layers with a ReLU activation and followed by max-pooling layers. This is very similar to the architectures that Yann LeCun advocated in the 1990s for image classification (with the exception of ReLU), but don't worry about the details, we will cover them all soon!

In [13]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu',
          input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

The model so far outputs 3D feature maps `(height, width, features)`.  On top of it we stick two fully-connected layers. We end the model with a single unit and a sigmoid activation, which is perfect for a binary classification. To go with it we will also use the `binary_crossentropy` loss to train our model.

In [15]:
model.add(Flatten()) # this converts our 3D feature maps to 1D feature vectors
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [16]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

Let's prepare our data. We will use `.flow_from_directory()` to generate batches of image data (and their labels) directly from our images in their respective folders.

In [17]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [18]:
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

In [19]:
batch_size = 16

In [20]:
# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        train_dir,  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

Found 2000 images belonging to 2 classes.


In [21]:
# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 800 images belonging to 2 classes.


We can now use these generators to train our model. Each epoch takes 15-20s on GPU and 300-400s on CPU. So it's definitely viable to run this model on CPU if you aren't in a hurry.

In [22]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)

Epoch 1/50
125/125 [==============================] - 17s 136ms/step - loss: 0.7434 - acc: 0.5145 - val_loss: 0.6905 - val_acc: 0.5012
Epoch 2/50
125/125 [==============================] - 15s 118ms/step - loss: 0.6970 - acc: 0.5605 - val_loss: 0.7124 - val_acc: 0.5750
Epoch 3/50
125/125 [==============================] - 15s 117ms/step - loss: 0.6659 - acc: 0.6270 - val_loss: 0.6327 - val_acc: 0.6025
Epoch 4/50
125/125 [==============================] - 15s 117ms/step - loss: 0.6628 - acc: 0.6270 - val_loss: 0.6310 - val_acc: 0.6550
Epoch 5/50
125/125 [==============================] - 15s 117ms/step - loss: 0.6334 - acc: 0.6440 - val_loss: 0.6040 - val_acc: 0.6450
Epoch 6/50
125/125 [==============================] - 15s 118ms/step - loss: 0.6120 - acc: 0.6800 - val_loss: 0.5862 - val_acc: 0.6813
Epoch 7/50
125/125 [==============================] - 15s 117ms/step - loss: 0.5938 - acc: 0.6880 - val_loss: 0.6003 - val_acc: 0.6975
Epoch 8/50
125/125 [==============================] - 1

This approach gets us to a validation accuracy of 0.79-0.81 after 50 epochs (a number that was picked arbitrarily --- because the model is small and uses aggressive dropout, it does not seem to be overfitting too much by that point). 

Note that the variance of the validation accuracy is fairly high, both because accuracy is a high-variance metric and because we only use 800 validation samples. A good validation strategy in such cases would be to do k-fold cross-validation, but this would require training k models for every evaluation round.

Let's save our model --- we will be using it to visualize the network later on:

In [23]:
model.save('cats_and_dogs_small.h5')

## Using Bottleneck Features

A more refined approach would be to leverage a network pre-trained on a large dataset. Such a network would have already learned features that are useful for most computer vision problems, and leveraging such features would allow us to reach a better accuracy than any method that would only rely on the available data.

We will use the VGG16 architecture, pre-trained on the ImageNet dataset ---a model we will learn about later. Because the ImageNet dataset contains several "cat" classes (persian cat, siamese cat, ...) and many "dog" classes among its total of 1000 classes, this model will already have learned features that are relevant to our classification problem. In fact, it is possible that merely recording the softmax predictions of the model over our data rather than the bottleneck features would be enough to solve our dogs vs. cats classification problem extremely well. However, the method we present here is more likely to generalize well to a broader range of problems, including problems featuring classes absent from ImageNet.

Here's what the VGG16 architecture looks like:

![](../../images/vgg16_original.png)

Our strategy will be as follow: we will only instantiate the convolutional part of the model, everything up to the fully-connected layers. We will then run this model on our training and validation data once, recording the output (the "bottleneck features" from the VGG16 model: the last activation maps before the fully-connected layers) as numpy arrays. Then we will train a small fully-connected model on top of these stored features.

The reason why we are storing the features offline rather than adding our fully-connected model directly on top of a frozen convolutional base and running the whole thing, is computational efficiency. Running VGG16 is expensive, especially if you're working on CPU, and we want to only do it once. Note that this prevents us from using data augmentation though.

In [ ]:
from keras import applications

model = applications.VGG16(include_top=False, weights='imagenet')

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)
train_generator = datagen.flow_from_directory(
        train_dir, 
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=None, # this means our generator will only yield batches of data, no labels
        shuffle=False) # our data will be in order, so all first 1000 images will be cats, then 1000 dogs

In [ ]:
bottleneck_features_train = model.predict_generator(
        train_generator, 2000 // batch_size)

In [ ]:
validation_generator = datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

In [ ]:
bottleneck_features_validation = model.predict_generator(
        validation_generator, 800 // batch_size)

We can now use these features to train a small fully-connected model:

In [ ]:
import numpy as np

# the features were saved in order, so recreating the labels is easy
train_labels = np.array([0] * 1000 + [1] * 1000)
validation_labels = np.array([0] * 400 + [1] * 400)

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=bottleneck_features_train.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Thanks to its small size, this model trains very quickly even on CPU (1s per epoch):

In [ ]:
model.fit(bottleneck_features_train, train_labels,
          epochs=50,
          batch_size=batch_size,
          validation_data=(bottleneck_features_validation, validation_labels))

We reach a validation accuracy of 0.90-0.91: not bad at all. This is definitely partly due to the fact that the base model was trained on a dataset that already featured dogs and cats (among hundreds of other classes).

We also save the weights of this model as we will use it for our next method!

In [ ]:
model.save_weights('bottleneck_fc_model.h5')

## Fine-tuning

To further improve our previous result, we can try to "fine-tune" the last convolutional block of the VGG16 model alongside the top-level classifier. Fine-tuning consist in starting from a trained network, then re-training it on a new dataset using very small weight updates. In our case, this can be done in 3 steps:

*  instantiate the convolutional base of VGG16 and load its weights
* add our previously defined fully-connected model on top, and load its weights
* freeze the layers of the VGG16 model up to the last convolutional block

![](../../images/vgg16_modified.png)

Note that:
* in order to perform fine-tuning, all layers should start with properly trained weights: for instance you should not slap a randomly initialized fully-connected network on top of a pre-trained convolutional base. This is because the large gradient updates triggered by the randomly initialized weights would wreck the learned weights in the convolutional base. In our case this is why we first train the top-level classifier, and only then start fine-tuning convolutional weights alongside it.
* we choose to only fine-tune the last convolutional block rather than the entire network in order to prevent overfitting, since the entire network would have a very large entropic capacity and thus a strong tendency to overfit. The features learned by low-level convolutional blocks are more general, less abstract than those found higher-up, so it is sensible to keep the first few blocks fixed (more general features) and only fine-tune the last one (more specialized features).
* fine-tuning should be done with a very slow learning rate, and typically with the SGD optimizer rather than an adaptative learning rate optimizer such as RMSProp. This is to make sure that the magnitude of the updates stays very small, so as not to wreck the previously learned features.

In [ ]:
vgg16 = applications.VGG16(input_shape=(150,150,3),
                           weights='imagenet',
                           include_top=False)

After instantiating the VGG base and loading its weights, we add our previously trained fully-connected classifier on top:

In [ ]:
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

Now its necessary to start with a fully-trained classifier including the top classifier to do fine-tuning. Thus we now load the weights we saved in the bottelneck method:

In [ ]:
top_model.load_weights('bottleneck_fc_model.h5')

Now we can add the model on top of the VGG16 convolutional base:

In [ ]:
from keras.models import Model

model = Model(inputs=vgg16.input, 
              outputs=top_model(vgg16.output))

We then proceed to freeze all convolutional layers up to the last convolutional block:

In [ ]:
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:15]:
    layer.trainable = False

In [ ]:
from keras import optimizers

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

Finally, we start training the whole thing, with a very slow learning rate:

In [ ]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
batch_size = 16

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
# fine-tune the model
model.fit_generator(
    train_generator,
    steps_per_epoch=2000 // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=800 // batch_size)

This approach gets us to a validation accuracy of 0.94 after 50 epochs. Great success!

Here are a few more approaches you can try to get to above 0.95:
* more aggresive data augmentation
* more aggressive dropout
* use of L1 and L2 regularization (also known as "weight decay")
* fine-tuning one more convolutional block (alongside greater regularization)